# ENTREGÁVEIS OFICIAIS:

Atendendo aos requisitos de coleta, ingestão, armazenamento, transformação, consumo e orquestração.

**1. Uso de ferramenta de ETL ou ingestão (como o PIG, Sqoop, Knime, Data Factory, Glue, Data Fusion ou semelhantes)**

- Copie o arquivo com os dados fornecidos pela empresa para o ambiente de armazenamento (data lake) escolhido, ex.: Hadoop HDFS, Google Cloud Storage, Azure Blob Storage, Amazon S3 ou equivalentes.

- Carregue os dados do data lake usando a ferramenta de ETL, prepare (limpeza, transformação, extração de campos, etc.) e estruture o arquivo para ser carregado em um ambiente de dados estruturados (Relacional ou Serverless SQL) para consumo. (ex.: Hive, Synapse, BigQuery, Redshift, Athena)
Para essa estruturação e preparação utilize PIG, Python, Spark, Glue, Knime, Data Factory ou semelhantes.

- Copie o arquivo preparado gerado para o ambiente local. O nome do arquivo deve ter o formato RM9999, onde RM9999 é o número do RM do representante do grupo.

- Usando SQOOP faça a carga dos dados preparados e estruturados no item 1.2 para o banco de dados Oracle.

- Execute consultas SQL que explorem os dados carregados, por exemplo, contagem de registros, médias por categoria, agrupamentos temporais, filtros complexos.

- Elabore um relatório com as decisões técnicas tomadas, dificuldades encontradas, screenshots das etapas resultados obtidos nas análises, conclusões com insights de negócio (ex: padrões, anomalias, oportunidades)

- Crie um arquivo PDF com as evidências das operações efetuadas nos passos anteriores. Inclua prints das execuções no terminal, códigos utilizados (em anexo), e queries executadas.


**2) Uso de ferramentas de dados não-relacionais estruturados (SQL) para consultas.**

Aqui o arquivo pós transformação e tratamento podem ser consumidos direto do data lake (storage) ou pode ser carregado através da ferramenta de ETL para o Ambiente Não Relacional.

- Importe o arquivo final da etapa anterior para um ambiente de análise como: Hive, Redshift, BigQuery, Synapse, Athena ou semelhante.

- Usando o (Hive, Synapse, Redshift, Athena, Big Query e etc) faça uma análise exploratória dos seus dados:
a) contagem de registros
b) visualize uma amostragem dos dados (as dez primeiras linhas).
c) descrição dos campos da tabela
d) contagem de valores distintos por campo chave
e) estatística descritiva (média, mínimo, máximo)
f) distribuição de frequência (contagem dos elementos de cada campo)

- Prepare um relatório de qualidade de dados. Inclua a taxa de campos nulos, tipos inconsistentes e campos redundantes.

- Elabore um pequeno dashboad (Power BI, Tableau, Apache Superset ou Matplotlib/Seaborn) com gráfico de tendencias, distribuição de categorias e KPIs relevantes.

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS bus")
spark.sql("CREATE SCHEMA IF NOT EXISTS bus.bronze")
spark.sql("CREATE SCHEMA IF NOT EXISTS bus.silver")
spark.sql("CREATE SCHEMA IF NOT EXISTS bus.gold")

In [0]:
spark.sql("USE CATALOG bus")
spark.sql("USE SCHEMA bronze")

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS bus.bronze.rw

In [0]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/bus/bronze/rw/dados_mockados.csv")

df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bus.bronze.ota_bus_ticket_sales")

In [0]:
%sql
SELECT * FROM bus.bronze.ota_bus_ticket_sales LIMIT 10

# Análise Exploratória dos Dados

In [0]:
%sql
SELECT COUNT(*) FROM bus.bronze.ota_bus_ticket_sales

In [0]:
%sql
SELECT * FROM bus.bronze.ota_bus_ticket_sales LIMIT 10

In [0]:
%sql
DESCRIBE TABLE bus.bronze.ota_bus_ticket_sales

In [0]:
df_ota_bus_ticket_sales = spark.table("bus.bronze.ota_bus_ticket_sales")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, DoubleType, IntegerType, LongType, FloatType

In [0]:
contagem_distinta = df_ota_bus_ticket_sales.select([
    countDistinct("nk_ota_localizer_id").alias("distinct_localizer_id"),
    countDistinct("fk_contact").alias("distinct_contacts"),
    countDistinct("place_origin_departure").alias("distinct_origin_departure")
])

display(contagem_distinta)

In [0]:
df_ota_bus = df_ota_bus_ticket_sales.withColumn("gmv_success", regexp_replace("gmv_success", ",", ".").cast("float"))

df_ota_bus_final = df_ota_bus.select(
    mean("gmv_success").alias("media_gmv"),
    min("gmv_success").alias("min_gmv"),
    max("gmv_success").alias("max_gmv"),
    mean("total_tickets_quantity_success").alias("media_tickets"),
    min("total_tickets_quantity_success").alias("min_tickets"),
    max("total_tickets_quantity_success").alias("max_tickets")
)

display(df_ota_bus_final)

In [0]:
df_ota_bus_ticket_sales.groupBy("fk_departure_ota_bus_company").count().orderBy("count", ascending=False).show()

In [0]:
df_ota_bus_ticket_sales.groupBy("place_origin_departure").count().orderBy("count", ascending=False).show()

# Relatório de Qualidade

In [0]:
%sql

-- DEIXAR O CÓDIGO DE QUALIDADE AQUI